This code was derived from https://www.steadforce.com/how-tos/lambada-method-how-to-use-data-augmentation-in-nlu

Housekeeping

In [1]:
!pip install wget
!pip install ktrain
!pip install transformers

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=d127f9b89dac86887ac3e76b6e948b8bc7cea253fab104edd40829885ce8d37c
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) 

In [2]:
import wget
import pandas as pd
from sklearn.utils import shuffle

In [3]:
wget.download('https://raw.githubusercontent.com/paulhimstedt/sarca_test/main/finetune_gpt.py')
wget.download('https://raw.githubusercontent.com/iabufarha/iSarcasmEval/main/train/train.En.csv')

'train.En.csv'

In [4]:
GLOBAL_GENERATE_MAX_LENGTH =50
GLOBALGENERATE_TOP_P = 0.92
NUMBER_OF_GENERATED_UTTERANCES_PER_INTENT = 300

Map labels

In [5]:
raw_df_train = pd.read_csv('train.En.csv')

# Define mapping
def map_sarcasm_to_intent(sarcasm):
    return "sarcastic" if sarcasm == 1 else "not sarcastic"

raw_df_train['intent'] = raw_df_train['sarcastic'].apply(map_sarcasm_to_intent)

# Rename the tweet column to utterance
raw_df_train.rename(columns={'tweet': 'utterance'}, inplace=True)
raw_df_train = raw_df_train[["intent", "utterance"]]
raw_df_train = shuffle(raw_df_train)
raw_df_train.to_csv('train.csv', index=False)
print(f"Train data length: {len(raw_df_train)}")

Train data length: 3468


In [6]:
!python finetune_gpt.py \
    --output_dir='/content/transformers/output' \
    --model_type=gpt2-medium \
    --model_name_or_path=gpt2-medium \
    --num_train_epochs=3 \
    --do_train \
    --train_data_file=/content/train.csv \
    --per_gpu_train_batch_size=24 \
    --block_size=10 \
    --gradient_accumulation_steps=1 \
    --line_by_line \
    --overwrite_output_dir

2023-12-21 03:01:09.052704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 03:01:09.052772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 03:01:09.054623: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 03:01:11.453059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/21/2023 03:01:13 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
config.json: 100% 718/718 [00:00<00:00, 3.15MB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 15.8MB/s

# Load and manually test model

In [7]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = TFGPT2LMHeadModel.from_pretrained('/content/transformers/output/', pad_token_id=tokenizer.eos_token_id)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [8]:
# Steadforce code settings produced this:
# Output:
# ----------------------------------------------------------------------------------------------------
# 0: sarcastic,@Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Sant
# 1: sarcastic,@Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Sant
# 2: sarcastic,@Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Sant
# 3: sarcastic,@Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Sant
# 4: sarcastic,@Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Sant
# 5: sarcastic,@Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Santandave1 @Sant

In [9]:
input_ids = tokenizer.encode('sarcastic,', return_tensors='tf')
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=128,
    top_k=12,
    top_p=0.82,
    num_return_sequences=10
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: sarcastic,Just finished the book 'The Magician's Shop' and it was a great read!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
1: sarcastic,@mikey_davies1 @RealMikeyDavies2 @RealMikeyDavies2 I’m sorry but I don’t understand the #GamerGate movement. I don’t understand the hate and negativity that has been directed at women and minorities in video games over the past year. I don’t understand the amount of time and effort that went into this #SuspendTheGamerGate hashtag and hashtag of women who were targeted by the #GamerGate hashtag. I don’t understand how anyone could support such a hate group and then
2: sarcastic,@DumbAsInTheSky @jeffreykraus @jeremykraus @dumbasilves @TheDumbAsInTheSky I'm gonna die of a broken heart because I'm too fucking stupid to realize that I’m a fucking retard 😭!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# prepare model for download

In [10]:
!zip -r -X gpt-2_tuned.zip transformers/output

  adding: transformers/output/ (stored 0%)
  adding: transformers/output/special_tokens_map.json (deflated 52%)
  adding: transformers/output/generation_config.json (deflated 24%)
  adding: transformers/output/model.safetensors (deflated 7%)
  adding: transformers/output/config.json (deflated 52%)
  adding: transformers/output/vocab.json (deflated 59%)
  adding: transformers/output/tokenizer_config.json (deflated 51%)
  adding: transformers/output/merges.txt (deflated 53%)
  adding: transformers/output/tokenizer.json (deflated 72%)
  adding: transformers/output/training_args.bin (deflated 50%)


In [11]:
def generate_utterances_df(n_generated, tokenizer, model, intent):
  input_ids = tokenizer.encode(intent + ',', return_tensors='tf')
  sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=GLOBAL_GENERATE_MAX_LENGTH,
    top_k=n_generated,
    top_p=GLOBALGENERATE_TOP_P,
    num_return_sequences=n_generated
  )

  list_of_intent_and_utterances = [
    (
        intent,
        tokenizer.decode(sample_output, skip_special_tokens=True)[len(intent)+1:]
    )
    for sample_output in sample_outputs
  ]

  return pd.DataFrame(list_of_intent_and_utterances, columns=['intent', 'utterance'])

Like in Tavor et al. , generate ten-times more samples as you want to finally use as augmentations (ca. 5400)

In [12]:
data_train = pd.read_csv(f'/content/train.csv')
intents = data_train["intent"].unique()

generated_utterances_df = pd.DataFrame(columns=['intent', 'utterance'])
while generated_utterances_df.shape[0] <= 5400:
  print(generated_utterances_df.shape[0])
  for intent in intents:
    print("Generating for intent " + intent)
    if intent == "sarcastic":
      utterances_for_intent_df = generate_utterances_df(NUMBER_OF_GENERATED_UTTERANCES_PER_INTENT, tokenizer, model, intent)
      generated_utterances_df = pd.concat([generated_utterances_df, utterances_for_intent_df], ignore_index=True)


0
Generating for intent not sarcastic
Generating for intent sarcastic
300
Generating for intent not sarcastic
Generating for intent sarcastic
600
Generating for intent not sarcastic
Generating for intent sarcastic
900
Generating for intent not sarcastic
Generating for intent sarcastic
1200
Generating for intent not sarcastic
Generating for intent sarcastic
1500
Generating for intent not sarcastic
Generating for intent sarcastic
1800
Generating for intent not sarcastic
Generating for intent sarcastic
2100
Generating for intent not sarcastic
Generating for intent sarcastic
2400
Generating for intent not sarcastic
Generating for intent sarcastic
2700
Generating for intent not sarcastic
Generating for intent sarcastic
3000
Generating for intent not sarcastic
Generating for intent sarcastic
3300
Generating for intent not sarcastic
Generating for intent sarcastic
3600
Generating for intent not sarcastic
Generating for intent sarcastic
3900
Generating for intent not sarcastic
Generating for i

In [13]:
generated_utterances_df.to_csv("generated.csv", index=False)